## Trading Strategy Simulation

#### Overview
* **Objective**: Simulate a trading strategy involving buying and selling stocks over a specific timeframe. The stocks are selected through a portfolio optimization process involving the CAPM model.


#### Trading Strategy

* **Initial Purchase**: Long position on selected stocks on day *t*.
* **Hold Period**: Hold the stocks for 7 trading days.
* **Sale**: Sell the stocks on day *t* + 7 trading days.
* **Reinvestment**: On the same day the stocks are sold, perform analysis and reinvest in new stocks.


#### Analysis Period:

*  **Historical Data Usage**: Use 30 days of market data (approximately 2 months) to analyze and select stocks for long positions


#### Simulation Duration:

*  **Total Period**: Conduct the simulation over a span 3.5 months approximately, adhering to the 7-day hold and reinvest cycle.

In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
from scipy.optimize import linprog

#### Defining a portfolio

These are POSSIBLE stocks. When solving the problem, most of them might have no weight in the selected portfolio.

In [ ]:
it_tickers= ['AAPL', 'MSFT', 'GOOGL', 'INTC', 'CSCO', 'IBM', 'ORCL', 'NVDA', 'ADBE', 'UBER']
energy_tickers= ['XOM', 'CVX', 'BP', 'SLB', 'E', 'EOG', 'KMI']
banking_tickers= ['JPM', 'BAC', 'WFC', 'C', 'GS', 'MS', 'UBS', 'HSBC', 'DB', 'AXP']
real_estate_tickers= ['SPG', 'CBRE', 'PLD', 'EQR', 'AVB', 'BXP', 'ARE', 'VNO', 'EQIX', 'CCI']
agriculture_food_tickers= ['MCD', 'KO', 'PEP', 'COST', 'MDLZ', 'KHC', 'SYY', 'ADM']
services_tickers= ['AMZN', 'WMT', 'HD', 'LOW', 'TGT', 'UNH', 'TJX', 'BKNG']

#Portfolio stocks
tickers= (it_tickers + energy_tickers + banking_tickers + real_estate_tickers +
         agriculture_food_tickers + services_tickers)

#### Downloading Data

In [ ]:
def getData(start_date, end_date):
    #Stocks data
    stocks_data= yf.download(tickers, start=start_date, end=end_date, progress=False)['Adj Close']
    stocks_data_open= yf.download(tickers, start=start_date, end=end_date, progress=False)['Open']
    stocks_data_close= yf.download(tickers, start=start_date, end=end_date, progress=False)['Close']
    #S&P 500 index data
    market_data= yf.download('^GSPC', start=start_date, end=end_date, progress=False)['Adj Close']
    #Data for beta0 using S&P 500 EFT
    sp500_data= yf.download('SPY', start=start_date, end=end_date, progress=False)['Adj Close']

    return stocks_data, stocks_data_open, stocks_data_close, market_data, sp500_data

#### Computing Betas and Mean Returns

In [ ]:
def get_meanreturns(buy_day, stocks_data):
    return stocks_data.iloc[buy_day-30:buy_day,:].pct_change().dropna().mean()

def get_betas(buy_day, stocks_data, market_data, sp500_data):
    n_stocks= stocks_data.shape[1]
    #Daily and mean returns
    stocks_returns= stocks_data.iloc[buy_day-30:buy_day,:].pct_change().dropna()

    market_returns= market_data.iloc[buy_day-30:buy_day].pct_change().dropna()
    #Stocks betas=cov(r,rm)/var(rm)
    betas= np.array([np.cov(stocks_returns.iloc[:,i],market_returns)[0, 1] / np.var(market_returns)
            for i in range(n_stocks)], dtype=float)
    #S&P 500 ETF beta
    sp500_returns= sp500_data.iloc[buy_day-30:buy_day].pct_change().dropna()
    beta500= np.cov(sp500_returns, market_returns)[0, 1] / np.var(market_returns)

    return betas, beta500

#### Matrix and Vector for the LP Problem

In [ ]:
def getA(betas):
    n= betas.shape[0]
    A= np.zeros((n+2,2*n + 1))
    #first two rows
    A[0,:n]= np.ones(n)
    A[1,:n]= betas
    A[1,n]= 1
    #The rest of the mtx is (I|0|I)
    A[2:,:n], A[2:,n+1:]= np.eye(n), np.eye(n)
    return A

def getd(n,b0):
    d= np.ones(n+2)
    d[1]= b0
    return d

#### Getting Stocks Allocation

In [ ]:
def get_StocksAlloc(buy_days, tolerance, stocks_data, market_data, sp500_data):
    Stocks=[]
    Weights=[]
    for i in range(len(buy_days)):
        Stocks.append([])
        Weights.append([])

    n_stocks= stocks_data.shape[1]
    for buy_day in buy_days:
        #Defining LP problem

        betas, beta500= get_betas(buy_day, stocks_data, market_data, sp500_data)
        #It is possible to have all betas greater than beta500*tolerance
        #and if that is the case there is no solution.
        b0= beta500*tolerance #if min(betas)<=beta500*1.15 else max(betas)

        c= np.zeros(2*n_stocks + 1)
        c[:n_stocks]= -get_meanreturns(buy_day, stocks_data)
        A= getA(betas)
        d= getd(n_stocks,b0)
        bounds= [(0, None)] * (2*n_stocks + 1)

        #Solving LP problem
        res= linprog(c, A_eq=A, b_eq=d,bounds=bounds, method='highs')
        for i in range(n_stocks):
            if res.x[i]>0:
                Stocks[int((buy_day-30)/7)].append(tickers[i])
                Weights[int((buy_day-30)/7)].append(res.x[i])
    return Stocks, Weights

#### Computing Returns

In [ ]:
def Portfolio_Performance(Stocks, Weights, buy_days, sell_days, stocks_data_close, stocks_data_open):
    Returns= np.zeros(len(Stocks))

    for i in range(len(Stocks)):
        buy_day= buy_days[i]
        sell_day= sell_days[i]

        for j in range(len(Stocks[i])):
            stock=Stocks[i][j]
            weight=Weights[i][j]
            #Sell at Open and buy at Close
            buy_price= stocks_data_close[stock][buy_day]
            sell_price= stocks_data_open[stock][sell_day]
            stock_return= (sell_price-buy_price)/buy_price

            Returns[i]+= weight*stock_return

    return Returns

def Total_Performance(Returns):
    return np.prod(Returns + 1) - 1

####  Backtesting

The strategy is being tested in 6 differente time windows.

In [ ]:
starts=['2022-07-01','2022-10-13','2023-01-27','2023-05-11','2023-08-24','2023-12-06']
ends=['2022-11-23','2023-03-10','2023-06-23','2023-10-05','2024-01-19','2024-05-02']
hist_data_range= 30 #30 tr days of historical data used
holding= 7 #7 tr days of holing

In [ ]:
ret=[]
ret500=[]
s=[]
for j in range(len(starts)):
    #Setting dates
    start_date= starts[j]
    end_date= ends[j]

    #Downloading data
    Data= getData(start_date, end_date)
    stocks_data, stocks_data_open, stocks_data_close= Data[:3]
    market_data, sp500_data = Data[3:]
    sp500_open= yf.download('SPY', start=market_data.index[30].date(), end=end_date, progress=False)['Open']
    sp500_close= yf.download('SPY', start=market_data.index[30].date(), end=end_date, progress=False)['Close']

    #Operation days
    total_trdays= stocks_data.shape[0]
    buy_days=list(range(hist_data_range,total_trdays-holding,holding))
    sell_days=list(range(hist_data_range + holding,total_trdays,holding))

    #Portfolio Optimization and Returns
    tolerance= 1 #Using the same beta of sp500
    Stocks, Weights= get_StocksAlloc(buy_days, tolerance, stocks_data, market_data, sp500_data)
    Returns= Portfolio_Performance(Stocks, Weights, buy_days, sell_days, stocks_data_close, stocks_data_open)
    total_return= Total_Performance(Returns)

    #S&P500 perfrormance
    sp500_return= (sp500_open[-1]-sp500_close[0])/sp500_close[0]

    #Saving Results
    ret.append(np.round(total_return*100,4))
    ret500.append(np.round(sp500_return*100,4))
    s.append(market_data.index[30].date())

In [ ]:
data={'Portfolio Return (%)':ret,'S&P 500 Return (%)':ret500,'Start Date':s, 'End Date':ends}
df=pd.DataFrame(data)
df

,Portfolio Return (%),S&P 500 Return (%),Start Date,End Date
0,14.1458,-7.5153,2022-08-15,2022-11-23
1,3.6571,-0.6437,2022-11-25,2023-03-10
2,9.2645,12.6090,2023-03-13,2023-06-23
3,-2.3558,-2.1718,2023-06-26,2023-10-05
4,19.2720,10.3529,2023-10-06,2024-01-19
5,38.4730,3.7088,2024-01-22,2024-05-02
